## Team Tracking System

We are developing a team tracking system for a medium-sized company to help managers keep track of their team's tasks, projects, and performance. The system aims to enhance communication, productivity, and decision-making among managers and their teams.


In [1]:
from secret_key import openai_key
import os
os.environ['OPENAI_API_KEY'] = openai_key

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router import MultiPromptChain

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

#### 1: Generate a title for team tracking system
Using LLM Chain to generate creative content

In [7]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name or title to describe \
    a {product}?"
)

In [9]:
chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
product = 'Team Tracking System'
chain.run(product)

'Team Performance Tracker'

#### 2: Generating marketing content for promoting the system's features using Sequential Chain

Generate marketing content for promoting the system's features and benefits to potential clients. Marketing content, including both a product description and a website page, using the following sub-chains:

Chain 1: Product Description Generation

The first chain focuses on generating a compelling product description for the Team Tracking System. This description highlights key features and benefits that the system offers to managers and organizations.

Chain 2: Website Page Description

The second chain is responsible for crafting an engaging website page description that summarizes the product description and entices potential clients to learn more about the system. The description should be concise and attention-grabbing.

We develop the Sequential Chain by connecting Chains 1 and 2 in the specified order.


In [12]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Generate a compelling product description for the {product}? \
     Highlight key features and benefits."
)
# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Craft an engaging website page description that summarizes the {product} description and entices potential clients to learn more about it."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
product = 'Team Tracking System'
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
The Team Tracking System is a powerful and user-friendly tool designed to help businesses manage their teams more effectively. With this system, you can easily keep track of your team members' progress, assign tasks, and monitor their performance, all in one convenient platform.

One of the key features of the Team Tracking System is its real-time tracking capabilities. This means that you can see exactly what your team members are working on at any given moment, ensuring that everyone is on the same page and working towards the same goals.

Another great feature of this system is its task management tools. With the Team Tracking System, you can assign tasks to specific team members, set deadlines, and track progress all in one place. This helps you stay organized and ensures that everyone is accountable for their part of the project.

The benefits of using the Team Tracking System are numerous. First and foremost, it can help increase pr

'Looking to manage your team more effectively? The Team Tracking System is the perfect solution! With real-time tracking capabilities and a user-friendly interface, this tool makes it easy to assign tasks, monitor performance, and track progress all in one convenient platform. Increase productivity, improve collaboration, and streamline your processes with the Team Tracking System. Try it today and see the difference it can make for your business!'

#### 3: Regular Sequential Chain for Team Tracking System

Processing and analyzing a **project's status** using a Regular Sequential Chain using the following chains:

Chain 1: Chain for Task Management

This chain summarizes task status from the given input.

Chain 2: Chain for Project Progress

Based on the task statuses and summaries understand the overall project progress. It should provide information about project status, timelines, and updates.

Chain 3: Chain for Team Performance

Based on project progress give overall team performance, such as team productivity, performance metrics, etc.

We combine these three chains into a Regular Sequential Chain.

Displaying the following:
1. Task Management
2. Project Progress
3. Team performance results

In [18]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: Task Management
first_prompt = ChatPromptTemplate.from_template(
    "Summarize the status of the given:"
    "\n\n{task}"
)
# chain 1: input= task and output=task_summary
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="task_summary"
                    )


In [19]:
# prompt template 2: Project Progress
second_prompt = ChatPromptTemplate.from_template(
    "Based on the {task_summary} understand the project progress. \
    It should provide information about project status, timelines, and updates."
)
# chain 2: input= task_summary and output=project_progress
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="project_progress"
                    )

In [20]:
# prompt template 3: Team Performance
third_prompt = ChatPromptTemplate.from_template(
    "Give the overall team performance, such as team productivity, performance metrics, etc from the :\n\n{project_progress}"
)
# chain 3: input= project_progress  and output=team_performance
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="team_performance"
                      )

In [23]:
# overall_chain: input= task
# and output= task_summary,project_progress,team_performance
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["task"],
    output_variables=["task_summary", "project_progress","team_performance"],
    verbose=True
)

In [24]:
task = "The Smart Office Implementation project aims to optimize workplace efficiency and sustainability through technology integration. Key tasks include \
Installation of IoT-based smart lighting and climate control systems. Deployment of occupancy sensors to monitor workspace utilization. \
Development of a mobile app for workspace customization. Integration of smart access controls for enhanced security. \
As of the latest update, the project is on schedule, with smart lighting and climate control systems operational. \
We anticipate significant energy savings and improved workspace utilization upon completion. The team remains committed to a smarter, more sustainable office environment."
overall_chain(task)



> Entering new SequentialChain chain...

> Finished chain.


{'task': 'The Smart Office Implementation project aims to optimize workplace efficiency and sustainability through technology integration. Key tasks include Installation of IoT-based smart lighting and climate control systems. Deployment of occupancy sensors to monitor workspace utilization. Development of a mobile app for workspace customization. Integration of smart access controls for enhanced security. As of the latest update, the project is on schedule, with smart lighting and climate control systems operational. We anticipate significant energy savings and improved workspace utilization upon completion. The team remains committed to a smarter, more sustainable office environment.',
 'task_summary': 'The Smart Office Implementation project aims to improve workplace efficiency and sustainability through technology integration. It includes installing smart lighting and climate control systems, deploying occupancy sensors, developing a mobile app for workspace customization, and inte


#### 4: Router Chain

We create three separate LLM Chains, each specialized for handling specific types of inquiries related to the team tracking system:

-- Employee attendance, Project Progress, and Inventory tracking.

Defined prompts for each category.

A Router Chain that routes manager inquiries to the appropriate subchain based on the content of the inquiry. The Router Chain has rules to identify keywords or phrases that indicate the inquiry category.

Provided a list of sample manager inquiries related to task management, project progress, and team performance. Included inquiries that cover a range of scenarios and issues.


In [25]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [26]:
employee_attendance_template = """You are the employee attendance manager. Your expertise lies in managing employee attendance records, tracking work hours, and addressing attendance-related inquiries. \
You can assist with attendance logging, leave approvals, and provide reports on employees' attendance history.

Here is an employee attendance inquiry:
{input}
"""

project_progress_template = """You are the project progress coordinator. Your role is to monitor and report on the progress of ongoing projects. \
You can provide updates on project timelines, milestones, and potential risks.

Here is a project progress inquiry:
{input}
"""

inventory_tracking_template = """You are the inventory tracking specialist. You excel in managing inventory records and tracking stock levels. \
You can assist with inquiries related to product availability, stock updates, and inventory reports.

Here is an inventory tracking inquiry:
{input}
"""

In [27]:
prompt_infos = [
    {
        "name": "employee attendance",
        "description": "To assist in employee attendance inquiries",
        "prompt_template": employee_attendance_template
    },
    {
        "name": "project progress",
        "description": "To assist in project progress inquiries",
        "prompt_template": project_progress_template
    },
    {
        "name": "inventory tracking",
        "description": "To assist in inventory tracking inquiries",
        "prompt_template": inventory_tracking_template
    }
]

In [28]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [29]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [30]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [33]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [36]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [37]:
employee_attendance_inquiries = ["Can you provide a report on the overall team's attendance for the past quarter?",
"What's the average employee attendance rate for our department this month?",
"I need to approve a leave request for John Smith. Can you confirm his available leave balance?",
"What's our policy regarding remote work attendance tracking?"]

project_progress_inquiries = ["Are there any outstanding issues affecting the 'Market Expansion' project's timeline?",
"What's the budget utilization status for the 'Client Onboarding' project?",
"Provide a summary of the 'Quality Assurance' project's progress and any potential bottlenecks.",
"We need a forecast for the 'Annual Sales Report' project completion date."]

inventory_tracking_inquiries = ["We're planning a promotional campaign. Can you confirm if we have enough product XYZ in stock?",
"Can you provide an inventory report showing the levels of raw materials for production?",
"What's our restocking strategy for office supplies, and how frequently do we reorder?",
"I need a report on the inventory turnover rates for each product category."]

In [38]:
chain.run(employee_attendance_inquiries[0])



> Entering new MultiPromptChain chain...
employee attendance: {'input': "Can you provide a report on the overall team's attendance for the past quarter?"}
> Finished chain.


"Yes, I can definitely provide a report on the overall team's attendance for the past quarter. I will need to access our attendance tracking system to gather the necessary data. Once I have compiled the information, I can generate a report that includes attendance rates, absences, tardiness, and any other relevant attendance-related data. Please let me know if there are any specific details you would like me to include in the report."

In [39]:
chain.run(employee_attendance_inquiries[1])



> Entering new MultiPromptChain chain...
employee attendance: {'input': "What's the average employee attendance rate for our department this month?"}
> Finished chain.


'Thank you for your inquiry. I can provide you with the average employee attendance rate for your department this month. May I know the name of your department and the specific dates you would like me to pull the attendance records for? This will help me provide you with accurate information.'

In [40]:
chain.run(employee_attendance_inquiries[2])



> Entering new MultiPromptChain chain...
employee attendance: {'input': "Can you provide me with John Smith's employee ID number?"}
> Finished chain.


"As the employee attendance manager, I can assist you with attendance-related inquiries, but I do not have access to employee ID numbers. Please reach out to the HR department or John Smith's supervisor for assistance with obtaining his employee ID number."

In [41]:
chain.run(project_progress_inquiries[0])



> Entering new MultiPromptChain chain...
project progress: {'input': "Are there any outstanding issues affecting the 'Market Expansion' project's timeline?"}
> Finished chain.


"As the project progress coordinator, I can confirm that there are currently no outstanding issues affecting the timeline of the 'Market Expansion' project. The project is progressing as planned and all milestones are being met on schedule. However, we will continue to monitor the project closely and will promptly address any potential risks that may arise."

In [42]:
chain.run(project_progress_inquiries[1])



> Entering new MultiPromptChain chain...
project progress: {'input': "What's the budget utilization status for the 'Client Onboarding' project?"}
> Finished chain.


"As the project progress coordinator, I can provide the following update on the budget utilization status for the 'Client Onboarding' project:\n\nCurrently, the project is within the allocated budget. However, we are closely monitoring the expenses and ensuring that we stay within the budget throughout the project's duration. We have implemented cost-saving measures where possible and are regularly reviewing the budget to identify any potential risks or areas where we can optimize spending. Overall, we are confident that we will successfully complete the project within the allocated budget."

In [43]:
chain.run(project_progress_inquiries[2])



> Entering new MultiPromptChain chain...
project progress: {'input': "Please provide an update on the progress of the 'Quality Assurance' project and identify any potential bottlenecks."}
> Finished chain.


"Thank you for your inquiry. As the project progress coordinator, I am happy to provide an update on the 'Quality Assurance' project.\n\nCurrently, the project is progressing according to the established timeline. The team has completed the initial planning phase and is now in the execution phase. The team has successfully identified the quality standards and requirements for the project and has developed a comprehensive quality assurance plan.\n\nHowever, there are a few potential bottlenecks that we have identified. One of the main challenges is the availability of resources, particularly in terms of personnel. The team is currently working with limited resources, which may impact the project's progress in the coming weeks.\n\nAdditionally, there may be potential risks associated with the implementation of the quality assurance plan. The team is working to mitigate these risks by conducting regular reviews and assessments of the plan.\n\nOverall, we are confident that the 'Quality As

In [44]:
chain.run(inventory_tracking_inquiries[0])



> Entering new MultiPromptChain chain...
inventory tracking: {'input': "We're planning a promotional campaign. Can you confirm if we have enough product XYZ in stock?"}
> Finished chain.


'Certainly! I can check our inventory records to see if we have enough product XYZ in stock. May I have the product code or SKU number for XYZ?'

In [45]:
chain.run(inventory_tracking_inquiries[1])



> Entering new MultiPromptChain chain...
inventory tracking: {'input': 'Can you provide an inventory report showing the levels of raw materials for production?'}
> Finished chain.


'Certainly! I can generate an inventory report that shows the current levels of all raw materials used in production. This report will include the quantity on hand, any pending orders, and any upcoming deliveries. Would you like me to send this report to you via email or provide it in a physical copy?'

In [46]:
chain.run(inventory_tracking_inquiries[2])



> Entering new MultiPromptChain chain...
inventory tracking: {'input': 'What is our current inventory level for office supplies and when was the last time we placed an order?'}
> Finished chain.


'As the inventory tracking specialist, I can provide you with the current inventory level for office supplies and the date of the last order. However, I would need access to our inventory management system to retrieve this information. Please provide me with the necessary login credentials or access to the system, and I will be happy to assist you with your inquiry.'